# 01 Baseline Model

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import pandas as pd
import numpy as np
from google.cloud import storage
import PIL
from matplotlib import pyplot as plt
from io import BytesIO
import tempfile
import os

In [6]:
tf.config.list_physical_devices('GPU')

2022-09-06 22:52:44.222832: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[]

2022-09-06 22:52:44.222859: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (simon-XPS-15-9575): /proc/driver/nvidia/version does not exist


In [7]:
query: str = "SELECT file_name, is_dirty FROM `plant-tech-dev.plant_tech_raw_metadata.base_labels` as t WHERE t.is_dirty_viable=1"
bucket_name: str = "plant-tech-raw"

In [8]:
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir.name)

/tmp/tmpwqtxpzig


## Lookup table

In [11]:
df_lookup = pd.read_gbq(
    query,
    project_id="plant-tech-dev"
)
print(f"Data Size: {len(df_lookup)}")
df_lookup.iloc[:5]


Data Size: 172


,file_name,is_dirty
0,hist_load/20220821_140712.jpg,1
1,hist_load/20220821_140733.jpg,1
2,hist_load/20220821_140741.jpg,1
3,hist_load/20220821_140800.jpg,1
4,hist_load/20220821_140956.jpg,1


In [21]:
df_lookup.set_index("file_name").T.to_dict()

/tmp/ipykernel_91722/1940744739.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df_lookup.set_index("file_name").T.to_dict()


{'hist_load/20220821_140712.jpg': {'is_dirty': 1},
 'hist_load/20220821_140733.jpg': {'is_dirty': 1},
 'hist_load/20220821_140741.jpg': {'is_dirty': 1},
 'hist_load/20220821_140800.jpg': {'is_dirty': 1},
 'hist_load/20220821_140956.jpg': {'is_dirty': 1},
 'hist_load/20220821_141008.jpg': {'is_dirty': 1},
 'hist_load/20220821_141020.jpg': {'is_dirty': 1},
 'hist_load/20220821_102529.jpg': {'is_dirty': 1},
 'hist_load/20220821_102534.jpg': {'is_dirty': 1},
 'hist_load/20220821_102543.jpg': {'is_dirty': 1},
 'hist_load/20220821_102545.jpg': {'is_dirty': 1},
 'hist_load/20220821_102552.jpg': {'is_dirty': 1},
 'hist_load/20220821_135926.jpg': {'is_dirty': 1},
 'hist_load/20220821_135928.jpg': {'is_dirty': 1},
 'hist_load/20220821_135930.jpg': {'is_dirty': 1},
 'hist_load/20220821_135938.jpg': {'is_dirty': 1},
 'hist_load/20220821_135940.jpg': {'is_dirty': 1},
 'hist_load/20220821_150608.jpg': {'is_dirty': 1},
 'hist_load/20220821_150703.jpg': {'is_dirty': 1},
 'hist_load/20220821_150725.jpg

## Pull Data  
Pull image data from bucket and dump to temporary directory with folder structure:  
```
{tmp_dir}/  
  dirty/
  clean/
```       

In [6]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)

In [7]:
%time

img_bytes = bucket.get_blob(df_lookup.file_name.iloc[0]).download_as_bytes()

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 9.54 µs


In [8]:
img = PIL.Image.open(BytesIO(img_bytes))

In [9]:
# np.asarray(img_array).shape

In [10]:
label_map = {
    1: "dirty",
    0: "clean"
}

In [11]:
for k,v in label_map.items():
    os.mkdir(dir := f"{temp_dir.name}/{v}")
    print(dir)

/tmp/tmpx7fwfqz7/dirty
/tmp/tmpx7fwfqz7/clean


In [12]:
def download_and_save(gcs_path, label, local_dir):
    file_name = gcs_path.split("/")[-1]
    img_bytes = bucket.get_blob(gcs_path).download_as_bytes()
    
    with open(f"{local_dir}/{label}/{file_name}", "wb") as img_file:
        img_file.write(img_bytes)

In [14]:
for index, row in df_lookup.iterrows():
    download_and_save(
        gcs_path=row[0],
        label=label_map[row[1]],
        local_dir=temp_dir.name
    )

In [38]:
seed = 42

train_dataset = keras.utils.image_dataset_from_directory(
    directory=temp_dir.name,
    batch_size=32,
    image_size=(256, 256),
    validation_split=0.3,
    subset="training",
    seed=seed
)

val_dataset = keras.utils.image_dataset_from_directory(
    directory=temp_dir.name,
    batch_size=32,
    image_size=(256, 256),
    validation_split=0.3,
    subset="validation",
    seed=seed
)

Found 156 files belonging to 2 classes.
Using 110 files for training.
Found 156 files belonging to 2 classes.
Using 46 files for validation.


## Baseline Model

In [16]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [17]:
# Using efficientnet TFHub example
model_name = "efficientnetv2-b2-21k"
model_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2"


In [45]:
input_pixels = (256, 256)
is_trainable = False

model = keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=input_pixels + (3, )),
    hub.KerasLayer(model_url, trainable=is_trainable),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(
        1,
        kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.build((None, ) + input_pixels + (3, ))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 1408)              8769374   
                                                                 
 dropout_4 (Dropout)         (None, 1408)              0         
                                                                 
 dense_4 (Dense)             (None, 1)                 1409      
                                                                 
Total params: 8,770,783
Trainable params: 1,409
Non-trainable params: 8,769,374
_________________________________________________________________


In [46]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy']
)

In [47]:
hist = model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset
).history

Epoch 1/100
4/4 [==============================] - 13s 2s/step - loss: 102512940845891584.0000 - accuracy: 0.3727 - val_loss: 70233684636073984.0000 - val_accuracy: 0.6304
Epoch 2/100
4/4 [==============================] - 6s 1s/step - loss: 180467894348414976.0000 - accuracy: 0.5909 - val_loss: 171344353019559936.0000 - val_accuracy: 0.4565
Epoch 3/100
4/4 [==============================] - 6s 1s/step - loss: 174215418397851648.0000 - accuracy: 0.5636 - val_loss: 57654021778833408.0000 - val_accuracy: 0.6304
Epoch 4/100
4/4 [==============================] - 6s 1s/step - loss: 70715725995573248.0000 - accuracy: 0.6182 - val_loss: 15818601848111104.0000 - val_accuracy: 0.6957
Epoch 5/100
4/4 [==============================] - 6s 1s/step - loss: 98554260899233792.0000 - accuracy: 0.5364 - val_loss: 39447328993574912.0000 - val_accuracy: 0.5000
Epoch 6/100
4/4 [==============================] - 7s 1s/step - loss: 105009356226953216.0000 - accuracy: 0.6182 - val_loss: 48030941393715200.00

### Cleanup

In [ ]:
temp_dir.cleanup()